In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import nibabel as nib
import cv2
from mlxtend.preprocessing import one_hot
import timeit
from random import shuffle
from MPC_tools import *
from layers import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


%matplotlib inline

# Making files path list

In [2]:
base_dir = '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data'

In [3]:
dir_path_list = []
filenames_list = []

for dirpath, dirnames, filenames in os.walk(base_dir):
    if 'model' in dirpath:
        dir_path_list.append(dirpath)

        temp_filenames_list=[]
        for i in range(len(filenames)):
            if 'sub' and 'npy' in filenames[i]:
                temp_filenames_list.append(filenames[i])
        filenames_list.append(temp_filenames_list)
        
dir_path_list = np.array(dir_path_list)
filenames_list = np.array(filenames_list)
filenames_list = np.sort(filenames_list, axis=1)

full_path_list = []

for i in range(dir_path_list.shape[0]):
    temp_path_list=[]
    for j in range(len(filenames_list[i])):
        temp_path = os.path.join(dir_path_list[i], filenames_list[i][j])
        temp_path_list.append(temp_path)
    full_path_list.append(temp_path_list)

full_path_list = np.array(full_path_list)

In [4]:
dir_path_list

array(['/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL',
       '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_Overall_FIR_SPM_SINGLE_TRIAL'],
      dtype='<U88')

In [5]:
filenames_list

array([['model02_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic002.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic003.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic004.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic005.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic007.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic008.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic009.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic010.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic011.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic012.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic013.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic014.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic015.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic016.npy',
        'model02_FIR_SPM_SINGLE_TRIAL_sub-semic018.npy',
        'model02_FIR_SPM_SINGLE

In [6]:
full_path_list

array([['/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic002.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic003.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic004.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic005.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy',
        '/home/cocoan-gpu1/D_drive/MPC/data/SEMIC/numpy_data/model02_FIR_SPM_SINGLE_TRIAL/model02_FIR_SPM_SINGLE_TRIAL_sub-semic007.npy',
        '/home/cocoan-gpu1/D_drive

# Loading the data

In [7]:
def load_data(subjec_num, shuffle=False, nan_to_num=False, one_hot_y=False, flatten_X=False):

    print("Loading {} data started ... ".format(filenames_list[0,subjec_num-1]))

    start = timeit.default_timer()

    X_stim = np.load(full_path_list[0,subjec_num-1])

    stop = timeit.default_timer()
    check_time = stop-start
    now = datetime.datetime.now()
    print("###  Duration Time  : {} minutes   {} seconds      ###".format(int(check_time)//60, int(check_time)%60))
    print("###  Excecuted Time : {}     ###".format(now))
    print('\n')


    print("Loading {} data started ... ".format(filenames_list[1,subjec_num-1]))

    start = timeit.default_timer()

    X_control = np.load(full_path_list[1,subjec_num-1])

    stop = timeit.default_timer()
    check_time = stop-start
    now = datetime.datetime.now()
    print("###  Duration Time  : {} minutes   {} seconds      ###".format(int(check_time)//60, int(check_time)%60))
    print("###  Excecuted Time : {}     ###".format(now))
    print('\n')
    
    print("{} data shape is {}".format(filenames_list[0,subjec_num-1], X_stim.shape))
    print("{} data shape is {}".format(filenames_list[1,subjec_num-1], X_control.shape))
    print('\n')

    print("Concatenating is started.")
    print("Shuffle is {}".format(shuffle))
    print("NaN to number is {}".format(nan_to_num))
    print("Y One hot transform is {}".format(one_hot_y))
    print("flatten X is {}".format(flatten_X))
    print("\n")

    stim_index_list = np.arange(0,X_stim.shape[0])
    
    if shuffle:
        np.random.shuffle(stim_index_list)
    
    X = np.concatenate((X_stim[stim_index_list[:X_control.shape[0]],:,:,:], X_control), axis=0)
    
    if nan_to_num:
        X = np.nan_to_num(X)
        
    if shuffle:
        index = np.arange(0,X.shape[0])
        np.random.shuffle(index)
        X = X[index,:,:,:]
    
    X = X[:,:,:,:,np.newaxis]
    
    if flatten_X:
        X = X.reshape((X.shape[0],-1))
    
    print("Concatenated data shape is : {}".format(X.shape))
    print("\n")

    
    Y_stim = np.ones(X_control.shape[0], dtype='i4')
    Y_control = np.zeros(X_control.shape[0], dtype='i4')
    Y = np.concatenate((Y_stim, Y_control), axis=0)
    
    if one_hot_y:
        from mlxtend.preprocessing import one_hot
        Y = one_hot(Y)
    
    if shuffle:
        Y = Y[index,:]
    
    print("Y shape is : {}".format(Y.shape))
    print("\n")

    return X, Y

# Making tensorflow graph

In [8]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def neural_net(x, num_classes=2):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1, activation=tf.nn.relu)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2, activation=tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    #out_layer = tf.layers.dense(layer_2, num_classes, activation=tf.nn.sigmoid)
    return layer_2


def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

def batch_norm(data, is_train, name=None):
    return tf.layers.batch_normalization(data, training=is_train, name=name)

In [9]:
# Network Parameters
n_hidden_1 = 128 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_classes = 2 

In [10]:
x = tf.placeholder(tf.float32,shape=[None, 592895])
y_true = tf.placeholder(tf.float32,shape=[None,num_classes])
is_train = tf.placeholder(tf.bool)
hold_prob = tf.placeholder(tf.float32)

In [11]:
dnn = neural_net(x, num_classes)

In [12]:
full_layer_one = tf.nn.relu(normal_full_layer(dnn,1024))
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [13]:
y_pred = normal_full_layer(full_one_dropout,2)

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
#cross_entropy = tf.reduce_mean(tf.losses.softmax_cross_entropy(y_true,y_pred))
#cross_entropy = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(y_true,y_pred))

#mse = tf.losses.mean_squared_error(y_true, y_pred)
learning_rate = 1e-5
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(cross_entropy)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# data count

total data = 4542

train data = 3633

validation data = 909

In [15]:
step_size = 30000

In [16]:
train_acc_list = []
train_entrop_list = []

test_acc_list = []
test_entrop_list = []

validation_acc_list = []
validataion_entrop_list = []

step_list = []

lowest_loss = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    subject_num = 1
    start = timeit.default_timer()

    for i in range(step_size):
        
        #Loading training and validation data sets
        if i%36 == 0 and i < step_size-44:
            X, Y = load_data(subject_num, shuffle=True, nan_to_num=True, one_hot_y=True, flatten_X=True)
            
            if subject_num<58:
                subject_num +=1   
            
            elif subject_num>=58:
                subject_num =1    

        #Load test data sets
        elif i == step_size-44:    
            validate_subjec_num = 58
            X, Y = load_data(validate_subjec_num, shuffle=True, nan_to_num=True, one_hot_y=True, flatten_X=True)

        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
        acc = tf.reduce_mean(tf.cast(matches,tf.float32))
        
        #training model
        if i < step_size-44:
            j = i%44
            _, train_entropy, train_accuracy = sess.run([train, cross_entropy, acc], 
                                                        feed_dict={x: X[j*100:j*100+100], 
                                                                   y_true: Y[j*100:j*100+100], 
                                                                   hold_prob: 0.5, is_train:True})

            validation_entropy, validation_accuracy= sess.run([cross_entropy, acc],
                                                  feed_dict={x: X[4400:4500], 
                                                             y_true: Y[4400:4500], 
                                                             hold_prob:1.0, is_train:False})
            if i%10 == 0:
                print("* * * * * * * * * * * * *")
                print('Currently on step {}'.format(i))
                print('##Train Cross entropy is: {}'.format(train_entropy))
                print('##Train Accuracy is: {}'.format(train_accuracy))
                print('\n')
                print('##Validation Cross entropy is: {}'.format(validation_entropy))
                print('##Validation Accuracy is: {}'.format(validation_accuracy))

                print("* * * * * * * * * * * * *")
                print('\n')
                
            train_acc_list.append(train_accuracy)
            train_entrop_list.append(train_entropy)
            
            validation_acc_list.append(validation_accuracy)
            validataion_entrop_list.append(validation_entropy)
            
            step_list.append(i)

        #testing model
        elif i > step_size-44:
            j = i%44
            test_entropy, test_accuracy= sess.run([cross_entropy, acc],
                                                  feed_dict={x:X[j*100:j*100+100], 
                                                             y_true:Y[j*100:j*100+100], 
                                                             hold_prob:1.0, is_train:False})

            test_acc_list.append(test_accuracy)
            test_entrop_list.append(test_entropy)
            
            step_list.append(i)
            
            if i%1 == 0:
                print("* * * * * * * * * * * * *")
                print('Currently on step {}'.format(i))
                print('##Test Cross entropy is: {}'.format(test_entropy))
                print('##Test Accuracy is: {}'.format(test_accuracy))

                print("* * * * * * * * * * * * *")
                print('\n')
            
        if lowest_loss == None or lowest_loss > validation_entropy:
            lowest_loss = validation_entropy
            saver.save(sess, "./model/MPC_dnn_basic_SEMIC_binary_classification_ver2.ckpt")
    
    
    train_acc_list = np.array(train_acc_list)
    train_entrop_list = np.array(train_entrop_list)
    
    test_entrop_list = np.array(test_entrop_list)
    test_acc_list = np.array(test_acc_list)
    
    validation_acc_list = np.array(validation_acc_list)
    validataion_entrop_list = np.array(validataion_entrop_list)
    
    step_list = np.array(step_list)
    
    stop = timeit.default_timer()
    check_time = stop-start
    now = datetime.datetime.now()

    print("!!!!Finished!!!!")
    print("###  Duration Time  : {} minutes   {} seconds      ###".format(int(check_time)//60, int(check_time)%60))
    print("###  Excecuted Time : {}     ###".format(now))

Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy data started ... 
###  Duration Time  : 1 minutes   13 seconds      ###
###  Excecuted Time : 2019-04-17 21:27:04.895537     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy data started ... 
###  Duration Time  : 0 minutes   33 seconds      ###
###  Excecuted Time : 2019-04-17 21:27:38.082395     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic001.npy data shape is (2271, 79, 95, 79)


Concatenating is started.
Shuffle is True
NaN to number is True
Y One hot transform is True
flatten X is True


Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * * * * * * * *
Currently on step 0
##Train Cross entropy is: 0.737925112247467
##Train Accuracy is: 0.4399999976158142


##Validation Cross entropy is: 0.6613181233406067
##Validation Accuracy is: 0.7699999809265137
* * * * * * * * * * * * *


* * * * * * * * 

Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy data started ... 
###  Duration Time  : 1 minutes   14 seconds      ###
###  Excecuted Time : 2019-04-17 21:40:05.578407     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy data started ... 
###  Duration Time  : 0 minutes   33 seconds      ###
###  Excecuted Time : 2019-04-17 21:40:38.995695     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic006.npy data shape is (2271, 79, 95, 79)


Concatenating is started.
Shuffle is True
NaN to number is True
Y One hot transform is True
flatten X is True


Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * * * * * * * *
Currently on step 180
##Train Cross entropy is: 0.43929585814476013
##Train Accuracy is: 0.800000011920929


##Validation Cross entropy is: 0.4127001166343689
##Validation Accuracy is: 0.7799999713897705
* * * * * * * * * * * * *


* * * * * * *

Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic011.npy data started ... 
###  Duration Time  : 1 minutes   14 seconds      ###
###  Excecuted Time : 2019-04-17 21:52:12.037915     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic011.npy data started ... 
###  Duration Time  : 1 minutes   15 seconds      ###
###  Excecuted Time : 2019-04-17 21:54:33.598708     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic012.npy data started ... 
###  Duration Time  : 0 minutes   33 seconds      ###
###  Excecuted Time : 2019-04-17 21:55:07.295571     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic012.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic012.npy data shape is (2271, 79, 95, 79)


Concatenating is started.
Shuffle is True
NaN to number is True
Y One hot transform is True
flatten X is True


Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * * * * * * * *
Currently on step 400
##Train Cross entropy is: 0.21

* * * * * * * * * * * * *
Currently on step 570
##Train Cross entropy is: 0.15339092910289764
##Train Accuracy is: 0.949999988079071


##Validation Cross entropy is: 0.2378828376531601
##Validation Accuracy is: 0.9399999976158142
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic018.npy data started ... 
###  Duration Time  : 1 minutes   16 seconds      ###
###  Excecuted Time : 2019-04-17 22:06:59.708196     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic018.npy data started ... 
###  Duration Time  : 0 minutes   34 seconds      ###
###  Excecuted Time : 2019-04-17 22:07:33.876198     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic018.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic018.npy data shape is (2271, 79, 95, 79)


Concatenating is started.
Shuffle is True
NaN to number is True
Y One hot transform is True
flatten X is True


Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * *

* * * * * * * * * * * * *
Currently on step 750
##Train Cross entropy is: 0.19467034935951233
##Train Accuracy is: 0.9200000166893005


##Validation Cross entropy is: 0.17666369676589966
##Validation Accuracy is: 0.9200000166893005
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic023.npy data started ... 
###  Duration Time  : 1 minutes   16 seconds      ###
###  Excecuted Time : 2019-04-17 22:19:18.216747     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic023.npy data started ... 
###  Duration Time  : 0 minutes   34 seconds      ###
###  Excecuted Time : 2019-04-17 22:19:52.609428     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic023.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic023.npy data shape is (2271, 79, 95, 79)


Concatenating is started.
Shuffle is True
NaN to number is True
Y One hot transform is True
flatten X is True


Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * *

...

* * * * * * * * * * * * *
Currently on step 2580
##Train Cross entropy is: 0.22569742798805237
##Train Accuracy is: 0.9100000262260437


##Validation Cross entropy is: 0.22704920172691345
##Validation Accuracy is: 0.8799999952316284
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 2590
##Train Cross entropy is: 0.2325267344713211
##Train Accuracy is: 0.8999999761581421


##Validation Cross entropy is: 0.2157338559627533
##Validation Accuracy is: 0.8799999952316284
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic015.npy data started ... 
###  Duration Time  : 1 minutes   15 seconds      ###
###  Excecuted Time : 2019-04-18 00:31:47.688650     ###


Loading model02_Overall_FIR_SPM_SINGLE_TRIAL_sub-semic015.npy data started ... 
###  Duration Time  : 0 minutes   34 seconds      ###
###  Excecuted Time : 2019-04-18 00:32:21.773317     ###


model02_FIR_SPM_SINGLE_TRIAL_sub-semic015.npy data shape is (6051, 79, 95, 79)
model02_Overall_FIR_SPM

Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * * * * * * * *
Currently on step 4290
##Train Cross entropy is: 0.22776439785957336
##Train Accuracy is: 0.8999999761581421


##Validation Cross entropy is: 0.283354252576828
##Validation Accuracy is: 0.8500000238418579
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4300
##Train Cross entropy is: 0.1300111711025238
##Train Accuracy is: 0.9599999785423279


##Validation Cross entropy is: 0.24393051862716675
##Validation Accuracy is: 0.8700000047683716
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4310
##Train Cross entropy is: 0.20437410473823547
##Train Accuracy is: 0.9100000262260437


##Validation Cross entropy is: 0.22580692172050476
##Validation Accuracy is: 0.8999999761581421
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic005.npy data started ... 
###  Duration Time  : 1 minutes   14 seconds      ###
###  Excecuted Time : 2

Concatenated data shape is : (4542, 592895)


Y shape is : (4542, 2)


* * * * * * * * * * * * *
Currently on step 4470
##Train Cross entropy is: 0.20584583282470703
##Train Accuracy is: 0.9100000262260437


##Validation Cross entropy is: 0.18746180832386017
##Validation Accuracy is: 0.9300000071525574
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4480
##Train Cross entropy is: 0.26891112327575684
##Train Accuracy is: 0.8999999761581421


##Validation Cross entropy is: 0.17073194682598114
##Validation Accuracy is: 0.9399999976158142
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4490
##Train Cross entropy is: 0.15577048063278198
##Train Accuracy is: 0.9300000071525574


##Validation Cross entropy is: 0.16255642473697662
##Validation Accuracy is: 0.9399999976158142
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic010.npy data started ... 
###  Duration Time  : 1 minutes   14 seconds      ###
###  Excecuted Time 

* * * * * * * * * * * * *
Currently on step 4650
##Train Cross entropy is: 0.1713458001613617
##Train Accuracy is: 0.9300000071525574


##Validation Cross entropy is: 0.17001865804195404
##Validation Accuracy is: 0.9399999976158142
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4660
##Train Cross entropy is: 0.1215021014213562
##Train Accuracy is: 0.9700000286102295


##Validation Cross entropy is: 0.1443561315536499
##Validation Accuracy is: 0.9599999785423279
* * * * * * * * * * * * *


* * * * * * * * * * * * *
Currently on step 4670
##Train Cross entropy is: 0.1785387396812439
##Train Accuracy is: 0.9100000262260437


##Validation Cross entropy is: 0.13658104836940765
##Validation Accuracy is: 0.9399999976158142
* * * * * * * * * * * * *


Loading model02_FIR_SPM_SINGLE_TRIAL_sub-semic015.npy data started ... 
###  Duration Time  : 1 minutes   15 seconds      ###
###  Excecuted Time : 2019-04-18 03:16:54.043246     ###


Loading model02_Overall_FIR_SPM_SING

MemoryError: 

# Visualize result

In [ ]:
result_list = pd.DataFrame({"train_acc_list":train_acc_list,"validation_acc_list":validation_acc_list,
                            "train_entrop_list":train_entrop_list,"validation_entrop_list":validataion_entrop_list,
                            "test_acc_list":test_acc_list, "test_entrop_list":test_entrop_list,
                            "step":step_list})
result_list.head()

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["train_acc_list"])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["validation_acc_list"])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["train_entrop_list"])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["validation_entrop_list"])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["test_acc_list"])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.lineplot(x = result_list["step"], y= result_list["test_entrop_list"])